# Setups for non-reactive simulations with LAMMPS

This notebook shows how to create an atomic configuration in a python script and how to generate input files for LAMMPS from it. As a simple example, we will set up ethane molecules and demonstrate the basic functionality of the `matscipy.opls` module.

`matscipy.opls.OPLSStructure` is a subclass of `ase.Atoms`. `OPLSStructure` objects can therefore be constructed and manipulated in the same way as `ase.Atoms` objects. The full documentation can be found in the __[ASE documentation](https://wiki.fysik.dtu.dk/ase/ase/atoms.html)__.

In [ ]:
import numpy as np
import ase
import ase.visualize
import matscipy.opls

a1 = matscipy.opls.OPLSStructure(
    'C2H6',
    positions = [[1.,  0.,  0.],
                 [2.,  0.,  0.],
                 [0.,  0.,  0.],
                 [1.,  1.,  0.],
                 [1., -1.,  0.],
                 [2.,  0.,  1.],
                 [2.,  0., -1.],
                 [3.,  0.,  0.]],
    cell = [5., 5., 5.]
)

ase.visualize.view(a1, viewer='ngl')

Alternatively, we can construct an `ase.Atoms` object and convert it to a `matscipy.opls.OPLSStructure` object:

In [ ]:
a2 = ase.Atoms(
    'C2H6',
    positions = [[1.,  0.,  0.],
                 [2.,  0.,  0.],
                 [0.,  0.,  0.],
                 [1.,  1.,  0.],
                 [1., -1.,  0.],
                 [2.,  0.,  1.],
                 [2.,  0., -1.],
                 [3.,  0.,  0.]],
    cell = [5., 5., 5.]
)
a2 = matscipy.opls.OPLSStructure(a2)

We can combine the two structures as we can for `ase.Atoms` objects.

In [ ]:
a = matscipy.opls.OPLSStructure(cell=[10., 10., 10.])
a1.translate([0., 4., 0.])
a.extend(a1)
a.extend(a2)
a.center()

ase.visualize.view(a, viewer='ngl')

Next, we specify atomic types. Note the difference between type and element. In this example we use two different types of hydrogen atoms.

In [ ]:
a.set_types(['C1', 'C1', 'H1', 'H1', 'H1', 'H2', 'H2', 'H2',
             'C1', 'C1', 'H1', 'H1', 'H1', 'H2', 'H2', 'H2'])

To perform a non-reactive simulation, manual specification of all pairs, angles, and dihedrals is required. Typically this involves searching the literature for appropriate parameters. This can be a tedious task. However, if the interactions present in the system are already known, this process can be greatly simplified. Lists of all existing interactions can be generated based on the distance of the atoms from each other. The maximum distances up to which two atoms are considered to interact can be read from a file which will typically look like this:

```
# Cutoffs
C1-C1 1.85
C1-H1 1.15
...
```
Here, we read these cutoffs from the file `cutoffs.in`:

In [ ]:
import matscipy.io.opls

cutoffs = matscipy.io.opls.read_cutoffs('cutoffs.in')
a.set_cutoffs(cutoffs)

bond_types, _ = a.get_bonds()
print(bond_types)

The same procedure applies to angles and dihedrals:

In [ ]:
angle_types, _ = a.get_angles()
print(angle_types)

In [ ]:
dih_types, _ = a.get_dihedrals()
print(dih_types)

Once all interaction parameters are known, they can be written to a file that looks like this:
```
# Element
C1 0.0028 3.50 -0.06
H1 0.0013 2.50  0.06
...

# Bonds
C1-C1 13.4 1.5
C1-H1 14.3 1.1
...

# Angles
C1-C1-H1 1.5 110.0
C1-C1-H2 1.1 115.0
...

# Dihedrals
H1-C1-C1-H2 0.0 0.0 0.016 0.0
...

# Cutoffs
C1-C1 1.85
...
```
Such a file can be used to generate the lists of all interactions and to create input files for LAMMPS:

In [ ]:
cutoffs, atom_data, bond_data, angle_data, dih_data = matscipy.io.opls.read_parameter_file('parameters.in')

a.set_cutoffs(cutoffs)
a.set_atom_data(atom_data)

After reading in all parameters, we can construct the lists of bonds, angles and dihedrals:

In [ ]:
a.get_bonds(bond_data)

In [ ]:
a.get_angles(angle_data)

In [ ]:
a.get_dihedrals(dih_data)

We have to call all these methods manually. The constructed lists are automatically stored in the `matscipy.OPLSStructure`-object, so there is no need to capture the return values at this point. Note that the construction of these lists can be *very* time-consuming for complex systems, especially the construction of the dihedrals list.

Now we can write the atomic structure, the potential definitions and a sample input script for LAMMPS (3 files in total). The input script contains a simple relaxation of the atomic position. You can use this file as a starting point to write scripts for more complex simulations.

In [ ]:
matscipy.io.opls.write_lammps('example', a)

Alternatively, atomic configurations can be read from any type of file ASE can __[read](https://wiki.fysik.dtu.dk/ase/ase/io/io.html#module-ase.io)__. Extended xyz-files are particularly useful for this purpose since they are human-readable and can contain information about the atom type. A typical extended XYZ file for non-reactive simulations will look like this
```
42
Lattice="10.0 0.0 0.0 0.0 10.0 0.0 0.0 0.0 10.0" Properties=species:S:1:pos:R:3:molid:I:1:type:S:1 pbc="F F F"
C        4.5       6.5       5.0       1        C1
H        4.5       5.5       5.0       1        H1
H        5.5       3.5       4.0       2        H2
...
```
The file should include the following columns: element (1 or 2 characters), x(float), y(float), z (float), molecule id (int), type (1 or 2 characters). A full description of the extended XYZ format can be found in the __[ASE documentation](https://wiki.fysik.dtu.dk/ase/ase/io/formatoptions.html#extxyz)__.

In [ ]:
b = matscipy.io.opls.read_extended_xyz('ethane.extxyz')

ase.visualize.view(b, viewer='ngl')